## Import library

In [38]:
import os
import subprocess
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yaml

import torch
from qucumber.nn_states import DensityMatrix
from qucumber.nn_states import ComplexWaveFunction
from qucumber.callbacks import MetricEvaluator
import qucumber.utils.unitaries as unitaries
import qucumber.utils.training_statistics as ts
import qucumber.utils.cplx as cplx
import qucumber.utils.data as data
from qucumber.observables import ObservableBase, to_pm1
from qucumber.observables.pauli import flip_spin
import qucumber

from qulacs.gate import Pauli

with open('./params_setting.yaml', 'r') as yml:
    params = yaml.safe_load(yml)
    
# quantum circuit parameter
n_qubit = params["circuit_info"]["n_qubit"]
n_data = params["circuit_info"]["n_data"]
each_n_shot = int(n_data / 3**n_qubit)
state_name = params["circuit_info"]["state_name"]
error_model = params["circuit_info"]["error_model"]
error_rate = params["circuit_info"]["error_rate"]
# RBM architecture parameter
n_visible_unit = params["architecture_info"]["n_visible_unit"]
n_hidden_unit = params["architecture_info"]["n_hidden_unit"] 
n_aux_unit = params["architecture_info"]["n_aux_unit"]
# train parameter
lr = params["train_info"]["lr"]
pbs = params["train_info"]["positive_batch_size"]
nbs = params["train_info"]["negative_batch_size"]
n_gibbs_step = params["train_info"]["n_gibbs_step"]
period = 1
epochs = params["train_info"]["n_epoch"]
lr_drop_epoch = params["train_info"]["lr_drop_epoch"]
lr_drop_factor = params["train_info"]["lr_drop_factor"]
seed = params["train_info"]["seed"]
# sampling parameter
n_sampling = params["sampling_info"]["n_sample"]
n_copy = params["sampling_info"]["n_copy"]
# data path info
train_data_path = f"./data/{noise_model}/error_prob_{100*error_rate}%/num_of_data_{n_data}"
ideal_state_path = f"./target_state"

# settings
## warnings
warnings.simplefilter('ignore')

## seaborn layout
sns.set()
sns.set_style("white")

## seed
def seed_settings(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    qucumber.set_random_seed(seed, cpu=True, gpu=False)

seed_settings(seed=seed)

## caluculate ideal state

In [42]:
# calculate ideal state
is_ideal_state_file = os.path.isfile(ideal_state_path + "/*.txt")
if is_ideal_state_file:
    print("ideal state data is exsisted !")
else:
    print("caluculate ideal state data ...")
    subprocess.run("python caluculate_ideal_state.py", shell=True)
    print("ideal state data is ready !")

False

## generate dataset

In [43]:
is_train_data_file = os.path.isfile(train_data_path + "/*.txt")
is_train_data_file

False

In [34]:
# generate train data
is_train_data_file = os.path.isfile(train_data_path + "/*.txt")
if is_train_data_file:
    print("train data is exsisted !")
else:
    print("generate directries & train data ...")
    os.makedirs(train_data_path, exist_ok = True)
    subprocess.run("python generate_dataset.py", shell=True)
    print("train data is ready !")

generate directries & train data ...


Traceback (most recent call last):
  File "/content/GitHub/NQS4QEM/Bell/generate_dataset.py", line 599, in <module>
    main()
  File "/content/GitHub/NQS4QEM/Bell/generate_dataset.py", line 584, in main
    ideal_state_vector_df["Re"] = ideal_state_vector_df["Re"].apply(lambda x: " ".join(x))
  File "/opt/conda/lib/python3.9/site-packages/pandas/core/series.py", line 4433, in apply
    return SeriesApply(self, func, convert_dtype, args, kwargs).apply()
  File "/opt/conda/lib/python3.9/site-packages/pandas/core/apply.py", line 1082, in apply
    return self.apply_standard()
  File "/opt/conda/lib/python3.9/site-packages/pandas/core/apply.py", line 1137, in apply_standard
    mapped = lib.map_infer(
  File "pandas/_libs/lib.pyx", line 2870, in pandas._libs.lib.map_infer
  File "/content/GitHub/NQS4QEM/Bell/generate_dataset.py", line 584, in <lambda>
    ideal_state_vector_df["Re"] = ideal_state_vector_df["Re"].apply(lambda x: " ".join(x))
TypeError: can only join an iterable


train data is ready !


## load dataset

In [44]:
meas_pattern = data.load_data_DM(train_data_path + "/measurement_pattern.txt")
meas_label = data.load_data_DM(train_data_path + "/measurement_label.txt")
meas_result = data.load_data_DM(train_data_path + "/measurement_result.txt")
ideal_rho_re = data.load_data_DM(ideal_state_path + "/rho_real.txt")
ideal_rho_im = data.load_data_DM(ideal_state_path + "/rho_imag.txt")

ValueError: could not convert string to float: 'X'